## Covid-19 Analysis - Part 4

**Which businesses/types of businesses are the primary offenders for these COVID violations?**

Dataset of hubNashville requests Allows you to see where violations occurred geographically, but it does not provide information in regard to the specific businesses that were reported

- Explore the businesses and types of businesses that have been reported

Warning: it will not be possible to match all violations to a business, and there will be ambiguities when trying to do this match, so do not spend all of your time on this task.

To attempt to answer this, you have been provided data from the Google Places API. Each file is formatted as a json. The values are as follows:

1. `mapped_location`: The mapped location from the hubNashville dataset
1. `address`: The address from the hubNashville dataset
1. `results`: The first five results from a Google Maps API nearbysearch, ranked by proximity to the Mapped Location. See here for more details on the fields in the results.

- Take these results and use them to match as many violations as possible to a business
- What do you find when you look into the types of businesses that have been reported for COVID violations?

In [1]:
import re
import requests

import datetime as dt
import ipywidgets as widgets
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

from io import StringIO
from ipywidgets import interact

In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 100)
sns.set()

## Importing Data - `hubNashville_cleaned.csv`

Importing covid19 data and checking the columns

In [3]:
covid_19 = pd.read_csv("../data/hubNashville_cleaned.csv")

Renaming the columns and dropping the following columns:
- `Unnamed: 0`
- `index`

In [4]:
covid_19 = covid_19.drop(columns=["Unnamed: 0", "index"])
covid_19.columns = ["req_num", "status", "req_type", "subreq_type", "add_subreq_type", "datetime_opened", "datetime_closed", "req_origin", "state_issue", "closed_when_created", "address", "city", "council_district", "zip", "latitude", "longitude", "mapped_loc", "month_num", "day_num", "month_day", "month_name", "weekday"]
covid_19.head()

,req_num,status,req_type,subreq_type,add_subreq_type,datetime_opened,datetime_closed,req_origin,state_issue,closed_when_created,address,city,council_district,zip,latitude,longitude,mapped_loc,month_num,day_num,month_day,month_name,weekday
0,444823,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,2020-08-14 02:38:00,2020-08-14 14:37:15,hubNashville Community,False,False,2045 Antioch Pike,ANTIOCH,28.0,37013,36.068546,-86.683898,"(36.0685457, -86.6838975)",8,14,8/14,Aug,Fri
1,479965,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,2020-10-13 11:48:40,2020-10-13 13:48:08,hubNashville Community,False,False,2800 Opryland Dr,NASHVILLE,15.0,37214,36.211269,-86.694302,"(36.211269429469425, -86.6943024067459)",10,13,10/13,Oct,Tue
2,392927,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,2020-06-16 00:35:20,2020-06-17 14:28:19,hubNashville Community,False,False,2607 12th Ave S,NASHVILLE,18.0,37204,36.123500,-86.790130,"(36.1235, -86.79013)",6,16,6/16,Jun,Tue
3,444512,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,2020-08-13 18:50:31,2020-08-14 14:22:11,hubNashville Community,False,False,3200 Clarksville Pike,NASHVILLE,2.0,37218,36.194163,-86.831394,"(36.1941628, -86.8313936)",8,13,8/13,Aug,Thu
4,444440,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,2020-08-13 18:03:09,2020-08-14 14:18:59,hubNashville Community,False,False,20 Grundy St,NASHVILLE,19.0,37203,36.158440,-86.788038,"(36.15843971160494, -86.7880380629013)",8,13,8/13,Aug,Thu


In [5]:
covid_19.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11632 entries, 0 to 11631
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   req_num              11632 non-null  int64  
 1   status               11632 non-null  object 
 2   req_type             11632 non-null  object 
 3   subreq_type          11632 non-null  object 
 4   add_subreq_type      11623 non-null  object 
 5   datetime_opened      11632 non-null  object 
 6   datetime_closed      11611 non-null  object 
 7   req_origin           11632 non-null  object 
 8   state_issue          11632 non-null  bool   
 9   closed_when_created  11632 non-null  bool   
 10  address              11258 non-null  object 
 11  city                 10653 non-null  object 
 12  council_district     10686 non-null  float64
 13  zip                  11632 non-null  int64  
 14  latitude             11354 non-null  float64
 15  longitude            11354 non-null 

## Analysis Decisions
- For date/time purposes, we will mainly adopt the `datetimme_opened`

## Importing Data - `google_places_results`